In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
## =================================================================
## = This code is to check the clearsky days
## = cloudiness = expected generation/clear-sky generation
## = if cloudiness >=0.8, it is a clear-sky day
## =================================================================

In [3]:
# Parameters
date_start = '2022-01-01'
date_end = '2023-11-01'

In [4]:
# read the clear-sky model generation for different PV sites
df_site = pd.read_csv('../preprocessed_data/PVsite_daily_20231016.csv')

df_site_clearsky = df_site.filter(like='EnergyYield.kWh.Daily')
split_columns = df_site_clearsky.columns.str.split(':').str[1]
df_site_clearsky.columns = split_columns
df_site_clearsky.insert(0, 'date', df_site['date'].values)
df_site_clearsky = df_site_clearsky[(df_site_clearsky['date'] >= date_start) & (df_site_clearsky['date'] <= date_end)]

# read the expected generation of different PV sites
df_site_expected = df_site.filter(like='Irrad.kWh.m2.Daily')
split_columns = df_site_expected.columns.str.split(':').str[1]
df_site_expected.columns = split_columns
df_site_expected.insert(0, 'date', df_site['date'].values)
df_site_expected = df_site_expected[(df_site_expected['date'] >= date_start) & (df_site_expected['date'] <= date_end)]

# Calculate the cloudiness
df_cloudiness = df_site_clearsky.copy()
df_cloudiness.iloc[:, 1:] = df_site_expected.iloc[:, 1:]/df_site_clearsky.iloc[:, 1:]
df_cloudiness.iloc[:, 1:] = df_cloudiness.iloc[:, 1:].astype(float)
df_cloudiness.dropna(how='all', axis=1, inplace=True)
df_cloudiness.iloc[:, 1:].fillna(axis=1, method='ffill', inplace=True)

df_clearday_sites = df_cloudiness.copy()
# Create a function to apply to each element in the DataFrame
def set_to_true_or_nan(x):
    if pd.isna(x):
        return x
    return x > 0.8

# Apply the function to the entire DataFrame
df_clearday_sites.iloc[:, 1:] = df_clearday_sites.iloc[:, 1:].applymap(set_to_true_or_nan)
df_clearday_sites

,date,SITE|65e61db2-2bb3-4480-b495-96064d5ed331,SITE|4dddc226-3464-4c95-aded-875e490a2f02,SITE|d830cf82-d943-404a-8396-9b22141538a8,SITE|c9fcf61f-c1a9-48d8-b2bb-c843dbd69119,SITE|09640f46-ed55-45be-b243-dbbe126fc994,SITE|166b83a9-ad51-4ad2-b853-fdb1aa936f6d,SITE|d317a168-4257-4187-9603-3d4834532bbb,SITE|11fde49c-5cd0-429c-8013-97c9a1c1462d,SITE|d159d534-3a20-4bce-9df0-5772d555d4c8,...,SITE|75493355-27b1-4676-91b8-dc598d46813b,SITE|bca93ed0-dcc7-4262-9bb0-320f25b0d93e,SITE|1b3255ea-c7ee-41c3-bfa1-b8748787de0c,SITE|14708719-4bec-45c7-8364-e3b765109de2,SITE|8ee3f4c9-483b-426a-bd77-2a7121b9bbcf,SITE|93dcd16f-e2e9-422c-9099-fa50c46998a5,SITE|ef7f758c-4612-4025-acc4-47cfec16891f,SITE|6577b77e-93fe-486b-bc95-b77c5ce6a18e,SITE|4229b257-cc70-4d03-8c0a-1f0d930d792e,SITE|49965a47-7a49-4866-a11f-06cb600ac967
1096,2022-01-01,NaN,True,NaN,True,NaN,NaN,True,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,True
1097,2022-01-02,NaN,True,NaN,True,NaN,NaN,True,NaN,NaN,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,False
1098,2022-01-03,NaN,True,NaN,True,NaN,NaN,True,NaN,NaN,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True
1099,2022-01-04,NaN,True,NaN,True,NaN,NaN,False,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,True
1100,2022-01-05,NaN,False,NaN,False,NaN,NaN,False,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761,2023-10-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1762,2023-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1763,2023-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1764,2023-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_clearday_sites.to_csv('../preprocessed_data/clearsky_sites.csv')